In [2]:
import pandas as pd
import numpy as np
import torch


In [21]:
path = '../data/training/training_solutions_rev1.csv' #path è universale
dataframe = pd.read_csv(path, sep=',') 
df = dataframe.drop(columns='GalaxyID')
df

,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,Class5.1,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,0.000000,...,0.000000,0.279952,0.138445,0.000000,0.000000,0.092886,0.000000,0.000000,0.0,0.325512
1,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,0.000000,...,0.018764,0.000000,0.131378,0.459950,0.000000,0.591328,0.000000,0.000000,0.0,0.000000
2,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,0.738832,0.238159,0.023009,0.000000,0.238159,0.000000,0.238159,0.000000,0.238159,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55415,0.510379,0.489621,0.000000,0.059207,0.430414,0.000000,0.430414,0.226257,0.204157,0.043458,...,0.000000,0.226257,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.226257
55416,0.901216,0.098784,0.000000,0.000000,0.098784,0.000000,0.098784,0.000000,0.098784,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
55417,0.202841,0.777376,0.019783,0.116962,0.660414,0.067245,0.593168,0.140022,0.520391,0.000000,...,0.000000,0.000000,0.090673,0.049349,0.000000,0.067726,0.000000,0.000000,0.0,0.072296
55418,0.091000,0.909000,0.000000,0.045450,0.863550,0.022452,0.841098,0.795330,0.068220,0.000000,...,0.000000,0.068398,0.318132,0.408799,0.227464,0.408799,0.090668,0.023065,0.0,0.045334


In [ ]:
def mapper(row,datafr=False):
    class1_3 = (row[:,0:3])
    class3_2 = (row[:,5:7])
    class4_2 = (row[:,7:9])
    class5_4 = (row[:,9:13])
    class7_3 = (row[:,15:18])
    class9_3 = (row[:,25:28])

    #normalizing class 4 and 5
    class4_2 = torch.where(class4_2.sum(dim=1, keepdim=True) != 0, 
                           class4_2 / class4_2.sum(dim=1, keepdim=True), 
                           class4_2)
    class5_4 = torch.where(class5_4.sum(dim=1, keepdim=True) != 0, 
                           class5_4 / class5_4.sum(dim=1, keepdim=True), 
                           class5_4)

    E0=class7_3[:,0]
    E3=class7_3[:,1]
    E6=class7_3[:,2]

    S0a_eon=class9_3[:,0]
    SB0a_eon=class9_3[:,1]
    Scd_eon=class9_3[:,2]

    SoB=class4_2[:,1]*class3_2[:,0]
    SoA=class4_2[:,1]*class3_2[:,1]

    SAa=class4_2[:,0]*class3_2[:,1]*class5_4[:,3]
    SAb=class4_2[:,0]*class3_2[:,1]*class5_4[:,2]
    SAc=class4_2[:,0]*class3_2[:,1]*class5_4[:,1]
    SAd=class4_2[:,0]*class3_2[:,1]*class5_4[:,0]

    SBa=class4_2[:,0]*class3_2[:,0]*class5_4[:,3]
    SBb=class4_2[:,0]*class3_2[:,0]*class5_4[:,2]
    SBc=class4_2[:,0]*class3_2[:,0]*class5_4[:,1]
    SBd=class4_2[:,0]*class3_2[:,0]*class5_4[:,0]

    A=class1_3[:,2]
    if datafr==True:x= pd.DataFrame({'E0':E0,
                     'E3': E3,
                     'E6': E6,
                     'S0a_eon': S0a_eon,
                     'SB0a_eon':SB0a_eon,
                     'Scd_eon': Scd_eon,
                     'SoB': SoB,
                     'SoA': SoA,
                     'SAa': SAa,
                     'SAb': SAb,
                     'SAc': SAc,
                     'SAd': SAd,
                     'SBa': SBa,
                     'SBb': SBb,
                     'SBc': SBc,
                     'SBd': SBd,
                     'A': A})
    else: x=torch.stack([E0,
                     E3,
                     E6,
                     S0a_eon,
                     SB0a_eon,
                     Scd_eon,
                     SoB,
                     SoA,
                     SAa,
                     SAb,
                     SAc,
                     SAd,
                     SBa,
                     SBb,
                     SBc,
                     SBd,
                     A],dim=1)
    return (x)
    


In [22]:
def mappy(tensor,datafr=False):

    '''
    This function takes as input a 1D tensor with the columns as structured in the .csv label file.
    It returns a new 1D tensor with a new set of labels that refer to the independent probabilities of 17 different classes.
    To see the result and the classes' name in a Pandas Dataframe set the parameter 'datafr' to True. 
    '''
    class1_3 = (tensor[0:3])
    class3_2 = (tensor[5:7])
    class4_2 = (tensor[7:9])
    class5_4 = (tensor[9:13])
    class7_3 = (tensor[15:18])
    class9_3 = (tensor[25:28])
    
    #normalizing class 4 and 5
    class4_2 = class4_2/class4_2.sum() if class4_2.sum() != 0 else class4_2
    class5_4 = class5_4/class5_4.sum() if class5_4.sum() != 0 else class5_4

    E0=class7_3[0]
    E3=class7_3[1]
    E6=class7_3[2]

    S0a_eon=class9_3[0]
    SB0a_eon=class9_3[1]
    Scd_eon=class9_3[2]

    SoB=class4_2[1]*class3_2[0]
    SoA=class4_2[1]*class3_2[1]

    SAa=class4_2[0]*class3_2[1]*class5_4[3]
    SAb=class4_2[0]*class3_2[1]*class5_4[2]
    SAc=class4_2[0]*class3_2[1]*class5_4[1]
    SAd=class4_2[0]*class3_2[1]*class5_4[0]
    
    SBa=class4_2[0]*class3_2[0]*class5_4[3]
    SBb=class4_2[0]*class3_2[0]*class5_4[2]
    SBc=class4_2[0]*class3_2[0]*class5_4[1]
    SBd=class4_2[0]*class3_2[0]*class5_4[0]

    A=class1_3[2]

    if datafr==True:x= pd.DataFrame([{'E0':E0,
                     'E3': E3,
                     'E6': E6,
                     'S0a_eon': S0a_eon,
                     'SB0a_eon':SB0a_eon,
                     'Scd_eon': Scd_eon,
                     'SoB': SoB,
                     'SoA': SoA,
                     'SAa': SAa,
                     'SAb': SAb,
                     'SAc': SAc,
                     'SAd': SAd,
                     'SBa': SBa,
                     'SBb': SBb,
                     'SBc': SBc,
                     'SBd': SBd,
                     'A': A}])
    else: x=torch.tensor([E0,
                     E3,
                     E6,
                     S0a_eon,
                     SB0a_eon,
                     Scd_eon,
                     SoB,
                     SoA,
                     SAa,
                     SAb,
                     SAc,
                     SAd,
                     SBa,
                     SBb,
                     SBc,
                     SBd,
                     A])
    return (x)


In [23]:
df_tensor=torch.Tensor(df.values)

In [24]:
x=mappy(df_tensor[0],datafr=True)
print(x)

               E0              E3          E6     S0a_eon    SB0a_eon  \
0  tensor(0.2015)  tensor(0.1817)  tensor(0.)  tensor(0.)  tensor(0.)   

      Scd_eon             SoB             SoA         SAa             SAb  \
0  tensor(0.)  tensor(0.0124)  tensor(0.1861)  tensor(0.)  tensor(0.3257)   

              SAc         SAd         SBa             SBb             SBc  \
0  tensor(0.0666)  tensor(0.)  tensor(0.)  tensor(0.0217)  tensor(0.0044)   

          SBd           A  
0  tensor(0.)  tensor(0.)  


In [25]:
print(x.sum(axis=1))

0    tensor(1.)
dtype: object


In [33]:
#x= mappy(df_tensor[0])
class4_2=df_tensor[0][0:3]
print(class4_2)
class4_2 = class4_2/class4_2.sum() if class4_2.sum() != 0 else class4_2


tensor([1.0001e+05, 3.8315e-01, 6.1685e-01])


In [11]:
x.sum()

tensor(2.6009)

In [ ]:
x['sum']=x.sum(axis=1)
x['sum']

IndexError: too many indices for tensor of dimension 1

# MAPPY PER DFs


In [3]:
path = '../data/test/test_solutions_rev1.csv' #path è universale
dataframe = pd.read_csv(path, sep=',') 
#df = dataframe.drop(columns='GalaxyID')

In [4]:
dataframe

,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,169764,0.502346,0.497654,0.000000,0.000000,0.497654,0.000000,0.497654,0.297856,0.199798,...,0.000000,0.168216,0.129640,0.000000,0.088114,0.042650,0.0,0.0,0.0,0.167092
1,539507,0.398953,0.553798,0.047249,0.000000,0.553798,0.186349,0.367449,0.000000,0.553798,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
2,229116,0.508830,0.491170,0.000000,0.031841,0.459329,0.000000,0.459329,0.000000,0.459329,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,428283,0.611838,0.337276,0.050885,0.000000,0.337276,0.157477,0.179799,0.166561,0.170715,...,0.000000,0.166561,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.166561
4,288410,0.093990,0.906010,0.000000,0.083025,0.822985,0.105038,0.717947,0.498948,0.324037,...,0.025536,0.254644,0.244304,0.000000,0.029248,0.264604,0.0,0.0,0.0,0.205096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6153,603354,0.782937,0.217063,0.000000,0.000000,0.217063,0.000000,0.217063,0.083594,0.133469,...,0.000000,0.000000,0.083594,0.000000,0.000000,0.083594,0.0,0.0,0.0,0.000000
6154,830615,0.679368,0.308373,0.012259,0.000000,0.308373,0.000000,0.308373,0.000000,0.308373,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
6155,525109,0.732937,0.242849,0.024215,0.000000,0.242849,0.000000,0.242849,0.141200,0.101649,...,0.000000,0.000000,0.000000,0.141200,0.000000,0.141200,0.0,0.0,0.0,0.000000
6156,100335,0.165002,0.834998,0.000000,0.235325,0.599673,0.148674,0.450999,0.378226,0.221447,...,0.078442,0.047278,0.236391,0.094556,0.141835,0.094556,0.0,0.0,0.0,0.141835


In [31]:
dfd = dataframe.iloc[:,1:3]
display(dfd)
dfd.sum(axis=1)

,Class1.1,Class1.2
0,0.502346,0.497654
1,0.398953,0.553798
2,0.508830,0.491170
3,0.611838,0.337276
4,0.093990,0.906010
...,...,...
6153,0.782937,0.217063
6154,0.679368,0.308373
6155,0.732937,0.242849
6156,0.165002,0.834998


0       1.000000
1       0.952751
2       1.000000
3       0.949114
4       1.000000
          ...   
6153    1.000000
6154    0.987741
6155    0.975786
6156    1.000000
6157    1.000000
Length: 6158, dtype: float64

In [60]:
def mappy_df(tensor):

    '''
    This function takes as input a 1D tensor with the columns as structured in the .csv label file.
    It returns a new 1D tensor with a new set of labels that refer to the independent probabilities of 17 different classes.
    To see the result and the classes' name in a Pandas Dataframe set the parameter 'datafr' to True. 
    '''
    class1_3 = np.array(tensor.iloc[:,1:4])
    class3_2 = np.array(tensor.iloc[:,6:8])
    class4_2 = np.array(tensor.iloc[:,8:10])
    class5_4 = np.array(tensor.iloc[:,10:14])
    class7_3 = np.array(tensor.iloc[:,16:19])
    class9_3 = np.array(tensor.iloc[:,26:29])
    
    #normalizing class 4 and 5
    class4_2 = np.where(class4_2.sum(axis=1, keepdims=True) != 0, 
                           class4_2 / class4_2.sum(axis=1, keepdims=True), 
                           class4_2)
    class5_4 = np.where(class5_4.sum(axis=1, keepdims=True) != 0, 
                           class5_4 / class5_4.sum(axis=1, keepdims=True), 
                           class5_4)
    E0=class7_3[:,0]
    E3=class7_3[:,1]
    E6=class7_3[:,2]

    S0a_eon=class9_3[:,0]
    SB0a_eon=class9_3[:,1]
    Scd_eon=class9_3[:,2]

    SoB=class4_2[:,1]*class3_2[:,0]
    SoA=class4_2[:,1]*class3_2[:,1]

    SAa=class4_2[:,0]*class3_2[:,1]*class5_4[:,3]
    SAb=class4_2[:,0]*class3_2[:,1]*class5_4[:,2]
    SAc=class4_2[:,0]*class3_2[:,1]*class5_4[:,1]
    SAd=class4_2[:,0]*class3_2[:,1]*class5_4[:,0]

    SBa=class4_2[:,0]*class3_2[:,0]*class5_4[:,3]
    SBb=class4_2[:,0]*class3_2[:,0]*class5_4[:,2]
    SBc=class4_2[:,0]*class3_2[:,0]*class5_4[:,1]
    SBd=class4_2[:,0]*class3_2[:,0]*class5_4[:,0]

    A=class1_3[:,2]

    x= pd.DataFrame({'GalaxyID':tensor['GalaxyID'],
                     'E0':E0,
                     'E3': E3,
                     'E6': E6,
                     'S0a_eon': S0a_eon,
                     'SB0a_eon':SB0a_eon,
                     'Scd_eon': Scd_eon,
                     'SoB': SoB,
                     'SoA': SoA,
                     'SAa': SAa,
                     'SAb': SAb,
                     'SAc': SAc,
                     'SAd': SAd,
                     'SBa': SBa,
                     'SBb': SBb,
                     'SBc': SBc,
                     'SBd': SBd,
                     'A': A})
    
    return x

In [62]:
x=mappy_df(dataframe)
x

/tmp/ipykernel_12449/2424779556.py:17: RuntimeWarning: invalid value encountered in divide
  class4_2 / class4_2.sum(axis=1, keepdims=True),
/tmp/ipykernel_12449/2424779556.py:20: RuntimeWarning: invalid value encountered in divide
  class5_4 / class5_4.sum(axis=1, keepdims=True),


,GalaxyID,E0,E3,E6,S0a_eon,SB0a_eon,Scd_eon,SoB,SoA,SAa,SAb,SAc,SAd,SBa,SBb,SBc,SBd,A
0,169764,0.000000,0.502346,0.000000,0.000000,0.000000,0.000000,0.000000,0.199798,0.092960,0.084919,0.119977,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,539507,0.331769,0.067184,0.000000,0.000000,0.000000,0.000000,0.186349,0.367449,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047249
2,229116,0.000000,0.296734,0.212096,0.031841,0.000000,0.000000,0.000000,0.459329,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,428283,0.029304,0.582534,0.000000,0.000000,0.000000,0.000000,0.079708,0.091006,0.023511,0.033731,0.031550,0.000000,0.020592,0.029543,0.027633,0.000000,0.050885
4,288410,0.000000,0.093990,0.000000,0.057489,0.000000,0.025536,0.041357,0.282680,0.012433,0.310182,0.112652,0.000000,0.001819,0.045381,0.016481,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6153,603354,0.729314,0.053623,0.000000,0.000000,0.000000,0.000000,0.000000,0.133469,0.015292,0.068302,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6154,830615,0.252356,0.427012,0.000000,0.000000,0.000000,0.000000,0.000000,0.308373,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012259
6155,525109,0.680525,0.052412,0.000000,0.000000,0.000000,0.000000,0.000000,0.101649,0.038584,0.044740,0.057876,0.000000,0.000000,0.000000,0.000000,0.000000,0.024215
6156,100335,0.000000,0.123752,0.041251,0.078442,0.078442,0.078442,0.054902,0.166545,0.000000,0.075378,0.066253,0.142823,0.000000,0.024849,0.021841,0.047082,0.000000
